In [1]:
!pip install librosa numpy scikit-learn


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
dataset_path = "/content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24"

In [22]:
import os
import librosa
import numpy as np
import pickle
from sklearn.mixture import GaussianMixture
from google.colab import files

In [23]:
emotion_labels = {
    "01": "Neutral",
    "02": "Calm",
    "03": "Happy",
    "04": "Sad",
    "05": "Angry",
    "06": "Fearful",
    "07": "Disgust",
    "08": "Surprised"
}

In [24]:
def extract_features(file_path, n_mfcc=13):
    try:
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfcc.T, axis=0)  # Take mean over time axis
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [28]:
X, y = [], []

for actor_folder in os.listdir(dataset_path):
    actor_path = os.path.join(dataset_path, actor_folder)

    if os.path.isdir(actor_path):
        for file in os.listdir(actor_path):
            file_path = os.path.join(actor_path, file)

            # Debug: Print file paths
            print(f"Processing: {file_path}")

            try:
                emotion_code = file.split("-")[2]  # Extract emotion code
                emotion = emotion_labels.get(emotion_code)

                if emotion:
                    features = extract_features(file_path)

                    # Debug: Check if features were extracted
                    if features is None:
                        print(f"⚠️ Feature extraction failed for: {file_path}")
                    else:
                        X.append(features)
                        y.append(emotion)

            except IndexError:
                print(f"⚠️ Skipping file due to filename format: {file}")
                continue

X = np.array(X)
y = np.array(y)

print(f"✅ Extracted {len(X)} feature vectors from the dataset.")


Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-01-01-01-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-02-02-01-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-02-02-02-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-01-02-02-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-01-01-02-02-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-03-02-01-02-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-01-02-01-24.wav
Processing: /content/drive/MyDrive/RAVDESS Emotional speech Audio/audio_speech_actors_01-24/Actor_24/03-01-02-01-01-02

In [29]:
gmm_models = {}

for emotion in np.unique(y):
    gmm = GaussianMixture(n_components=4, max_iter=200, covariance_type='diag', random_state=42)

    emotion_data = X[y == emotion]  # Select data for this emotion
    gmm.fit(emotion_data)

    gmm_models[emotion] = gmm

print("✅ Model Training Complete!")


✅ Model Training Complete!


In [30]:
# Save trained GMM models
with open("/content/drive/MyDrive/gmm_models.pkl", "wb") as f:
    pickle.dump(gmm_models, f)

print("✅ GMM Models Saved Successfully!")

# Load models (for future use)
with open("/content/drive/MyDrive/gmm_models.pkl", "rb") as f:
    gmm_models = pickle.load(f)


✅ GMM Models Saved Successfully!


In [31]:
# Upload a .wav file for testing
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Extract features and predict emotion
features = extract_features(file_name).reshape(1, -1)
scores = {emotion: gmm.score(features) for emotion, gmm in gmm_models.items()}
predicted_emotion = max(scores, key=scores.get)

print(f"🎭 Predicted Emotion: {predicted_emotion}")


Saving 03-01-02-01-01-01-21.wav to 03-01-02-01-01-01-21.wav
🎭 Predicted Emotion: Neutral
